### --- Day 10: Pipe Maze ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/10

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2023, 10);
WriteLines(inputLines);

Loading puzzle file: Day10.txt
Total lines: 140
Max line length: 140

7-LJ7.F-F77FF-77FJ-J-F7FF|777F7--..JJ.F.7.|.F-J7-J777F7FF77F|.|7L-7.F-|7F7FF-J7LF|.7--7J-F.F--7-L--77.|F-J77F7F|-F-F7-J-FFF---F--77|FF7--L7.
L7.F--J.L7J7|---FJ-|JLF77L7J7F|-J7|J.FLLJ.FFF7|F7L7-J7LF7.-|L-77-LL7J.|.L-7JFJ7-7J7L-LFJF-7-|7|-|FL--J|L77L-J|7|F|-7J.|LLJJL|.|JF7-|-|||L|-F
F|--7.L-7|.-L-|.|7-LL-J|L-|.FJJ7F-..F|.L|FLJL7F7|F.L-|-7LJL|JL77-.|7|7.J.LF-7FJ7JF-J||J|FL77LJJ-7-77||F--7JL-7-LJJF--77LLF7F7-|.JJ|||JFF-|JF
FJ|L7-FL-L7J|FL-|LJFL7-F|LJ--J.F-7F-7L7--7.|F7J|F|7.L||F7J-J.FFJ.LLF-7F..-|--JLJF7J.LF.LJ|L-JJ.|.LJ-LFF7|L7-L|-LF7|F|7--7LJF77|7|LF|7FJ|L|F|
|LF7L-7J||LF-FJ.|7FF-F--J-J-L-|JJLL.J.||L7-FF7|L-L-7F77L77FJFFLJ7|JL7F-77-L-JL--LJJ7L-77LJJJ|.FF-FJLL|L--L7F|JFLF7FLJ7FJJJFL7JJ77-7|LJL|.J-F


In [4]:
record Point(int x, int y)
{
    public Point Up() => new Point(x, y - 1);
    public Point Down() => new Point(x, y + 1);
    public Point Left() => new Point(x - 1, y);
    public Point Right() => new Point(x + 1, y);
}

record Piece(Point point, char shape);

In [5]:
IEnumerable<Piece> ParsePieces(string line, int rowNum) {
    return line.Select((ch, i) => new Piece(new(i, rowNum), ch));
}

In [6]:
var testInput = """
.....
.S-7.
.|.|.
.L-J.
.....
""";
var testInputLines = testInput.Split('\n');

In [7]:
Dictionary<Point, Piece> GetPieceMap(string[] inputLines) {
    return inputLines.SelectMany((line, row) => ParsePieces(line, row)).ToDictionary(p => p.point);
}

In [8]:
var testPieceMap = GetPieceMap(testInputLines);
var testStartPiece = testPieceMap.Values.Where(p => p.shape == 'S').Single();
Console.WriteLine(testStartPiece);

Piece { point = Point { x = 1, y = 1 }, shape = S }


In [9]:
Point[] Traverse(Piece current) {
    var p = current.point;

    return current.shape switch {
        // | is a vertical pipe connecting north and south.
        '|' => [p.Up(), p.Down()],
        // - is a horizontal pipe connecting east and west.
        '-' => [p.Left(), p.Right()],
        // L is a 90-degree bend connecting north and east.
        'L' => [p.Up(), p.Right()],
        // J is a 90-degree bend connecting north and west.
        'J' => [p.Up(), p.Left()],
        // 7 is a 90-degree bend connecting south and west.
        '7' => [p.Down(), p.Left()],
        // F is a 90-degree bend connecting south and east.
        'F' => [p.Down(), p.Right()],
        // . is ground; there is no pipe in this tile.
        '.' => [],
        // S is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.
        'S' => [p.Up(), p.Down(), p.Left(), p.Right()],
        _ => throw new Exception($"Unexpected piece shape {current.shape}")
    };
}

In [10]:
// The graph is too deep to recurse, so let's iterate instead

// We know that the pipe is a single connected string, so we don't need to
// consider alternative paths

List<Piece> IterateLoop(Dictionary<Point, Piece> pieceMap, HashSet<Point> visited, Piece piece) 
{
    // Starting point S has 4 potential inputs but is guaranteed to have 2 only.
    // So we need to find one connected input as that's guaranteed to be a part of
    // the connected loop

    var upDownLeftRight = Traverse(piece);
    var connectedPoint = upDownLeftRight.Where(x => Traverse(pieceMap[x]).Any(p => p == piece.point)).First();
    var connectedPiece = pieceMap[connectedPoint];
    Console.WriteLine($"Found pipe entry at {connectedPiece}");
    
    var i = 0;

    List<Piece> result = [piece];
    visited.Add(piece.point);

    var currentPiece = connectedPiece;
    while (true) {
        result.Add(currentPiece);
        visited.Add(currentPiece.point);

        // Console.WriteLine($"Adding {currentPiece}");

        var nextPoint = Traverse(currentPiece).Where(t => !visited.Contains(t)).FirstOrDefault(); // Pick a direction on the first piece

        if (nextPoint == null) {
            Console.WriteLine("No more next piece!");
            return result;
        }

        currentPiece = pieceMap[nextPoint];

        if (++i > 100000) {
            throw new Exception("Exceeded max iterations");
        }
    }
}


In [11]:
var test = IterateLoop(testPieceMap, new(), testStartPiece);

// In this example, the farthest point from the start is 4 steps away.
Console.WriteLine(test.Count / 2);

foreach (var t in test) {
    Console.WriteLine(t);
}


Found pipe entry at Piece { point = Point { x = 1, y = 2 }, shape = | }
No more next piece!
4
Piece { point = Point { x = 1, y = 1 }, shape = S }
Piece { point = Point { x = 1, y = 2 }, shape = | }
Piece { point = Point { x = 1, y = 3 }, shape = L }
Piece { point = Point { x = 2, y = 3 }, shape = - }
Piece { point = Point { x = 3, y = 3 }, shape = J }
Piece { point = Point { x = 3, y = 2 }, shape = | }
Piece { point = Point { x = 3, y = 1 }, shape = 7 }
Piece { point = Point { x = 2, y = 1 }, shape = - }


In [12]:
var testInput2 = """
7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ
""";
var testInputLines2 = testInput2.Split('\n');
var testPieceMap2 = GetPieceMap(testInputLines2);
var testStartPiece2 = testPieceMap2.Values.Where(p => p.shape == 'S').Single();
Console.WriteLine(testStartPiece2);

var test2 = IterateLoop(testPieceMap2, new(), testStartPiece2);

// Max distance for this one is 8
Console.WriteLine(test2.Count / 2);

foreach (var t in test2) {
    Console.WriteLine(t);
}

Piece { point = Point { x = 0, y = 2 }, shape = S }
Found pipe entry at Piece { point = Point { x = 0, y = 3 }, shape = | }
No more next piece!
8
Piece { point = Point { x = 0, y = 2 }, shape = S }
Piece { point = Point { x = 0, y = 3 }, shape = | }
Piece { point = Point { x = 0, y = 4 }, shape = L }
Piece { point = Point { x = 1, y = 4 }, shape = J }
Piece { point = Point { x = 1, y = 3 }, shape = F }
Piece { point = Point { x = 2, y = 3 }, shape = - }
Piece { point = Point { x = 3, y = 3 }, shape = - }
Piece { point = Point { x = 4, y = 3 }, shape = J }
Piece { point = Point { x = 4, y = 2 }, shape = 7 }
Piece { point = Point { x = 3, y = 2 }, shape = L }
Piece { point = Point { x = 3, y = 1 }, shape = | }
Piece { point = Point { x = 3, y = 0 }, shape = 7 }
Piece { point = Point { x = 2, y = 0 }, shape = F }
Piece { point = Point { x = 2, y = 1 }, shape = J }
Piece { point = Point { x = 1, y = 1 }, shape = F }
Piece { point = Point { x = 1, y = 2 }, shape = J }


In [13]:
var pieceMap = GetPieceMap(inputLines);
var startPiece = pieceMap.Values.Where(p => p.shape == 'S').Single();
Console.WriteLine($"Start piece is {startPiece}");

var result = IterateLoop(pieceMap, new(), startPiece);

// Find the single giant loop starting at S. How many steps along the loop does
// it take to get from the starting position to the point farthest from the
// starting position?

var part1Answer = result.Count / 2;
Console.WriteLine(part1Answer);

Start piece is Piece { point = Point { x = 93, y = 24 }, shape = S }
Found pipe entry at Piece { point = Point { x = 93, y = 25 }, shape = | }
No more next piece!
6956


In [14]:
// 6956 is correct!
Ensure(6956, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/10

In [16]:
// Looks like we actually need to infer the exact shape of S so we can properly
// enclose the shape

char InferShape(Dictionary<Point, Piece> pieceMap, Piece sPiece) 
{
    bool hasStart(Point p) => Traverse(pieceMap[p]).Any(p => p == sPiece.point);

    var up = hasStart(sPiece.point.Up());
    var down = hasStart(sPiece.point.Down());
    var left = hasStart(sPiece.point.Left());
    var right = hasStart(sPiece.point.Right());

    var result = (up, down, left, right) switch {
        (true, true, false, false) => '|',
        (true, false, true, false) => 'J',
        (true, false, false, true) => 'L',
        (false, false, true, true) => '-',
        (false, true, true, false) => '7',
        (false, true, false, true) => 'F',
        _ => throw new Exception("Unexpected connections")
    };

    return result;
}

In [17]:
// We can calculate the inside pieces by "drawing a line" throw a row. If we
// have passed an odd number of walls, the following pieces are inside. Conversely
// if we have passed an even number of walls, the pieces are outside. We need to be
// careful with curved pieces: it only counts as a wall if it does not turn back on
// itself!

class Scanner(List<Piece> loopPieces)
{
    public bool IsInside { get; private set; } = false;

    public List<Piece> InsidePieces { get; } = new();

    HashSet<Piece> loopPieces = new(loopPieces);

    Piece transitionPiece = null;

    public Scanner ScanOne(Piece piece) 
    {
        if (!loopPieces.Contains(piece)) {
            // Not part of the loop. Continue as in/out
            if (IsInside) {
                InsidePieces.Add(piece);
            }
        } else {
            // We hit a loop piece

            switch (piece.shape) {
                case '|':
                    IsInside = !IsInside;
                break;
                case '-':
                    // Part of a transition
                    if (transitionPiece == null) {
                        throw new Exception("Expected transition piece");
                    }
                break;
                case 'L':
                    transitionPiece = piece;
                break;
                case 'J':
                    if (transitionPiece.shape == 'F') {
                        // Transitioned into a wall
                        IsInside = !IsInside;
                    }
                    transitionPiece = null;
                break;
                case '7':
                    if (transitionPiece.shape == 'L') {
                        // Transitioned into a wall
                        IsInside = !IsInside;
                    }
                    transitionPiece = null;
                break;
                case 'F':
                    transitionPiece = piece;
                break;
                case '.':
                    throw new Exception("Should not happen");
                case 'S':
                    throw new Exception("Need to remove S");
                default:
                    throw new Exception($"Unexpected piece of wall: {piece}");
            }
        }

        return this;
    }
}

In [18]:
string part2Input = """
...........
.S-------7.
.|F-----7|.
.||OOOOO||.
.||OOOOO||.
.|L-7OF-J|.
.|II|O|II|.
.L--JOL--J.
.....O.....
""";
var part2InputLines = part2Input.Split('\n');
var part2PieceMap = GetPieceMap(part2InputLines);
var part2PieceStart = part2PieceMap.Values.Where(p => p.shape == 'S').Single();
var part2Result = IterateLoop(part2PieceMap, new(), part2PieceStart);

var part2StartShape = InferShape(part2PieceMap, part2PieceStart);
Console.WriteLine(part2StartShape);
// Replace start piece
var newStartPiece = new Piece(part2PieceStart.point, part2StartShape);
part2PieceMap[part2PieceStart.point] = newStartPiece;
part2Result = IterateLoop(part2PieceMap, new(), newStartPiece);

var part2Scanner = new Scanner(part2Result);
var part2TestRow = Enumerable.Range(0, part2InputLines[0].Length).Select(x => part2PieceMap[new Point(x, 6)]).ToArray();
var part2TestRowScanned = part2TestRow.Aggregate(part2Scanner, (a, b) => a.ScanOne(b));
Console.WriteLine($"Inside pieces of this row: {part2TestRowScanned.InsidePieces.Count()}");

Found pipe entry at Piece { point = Point { x = 1, y = 2 }, shape = | }
No more next piece!
F
Found pipe entry at Piece { point = Point { x = 1, y = 2 }, shape = | }
No more next piece!
Inside pieces of this row: 4


In [19]:
var startShape = InferShape(pieceMap, startPiece);
Console.WriteLine($"Inferred start shape is {startShape}. Will replace");
// Replace start Piece
var newStartPiece = new Piece(startPiece.point, startShape);
pieceMap[startPiece.point] = newStartPiece;

var result2 = IterateLoop(pieceMap, new(), newStartPiece);

var rows = Enumerable.Range(0, inputLines.Length);
var cols = Enumerable.Range(0, inputLines[0].Length);
var rowCols = rows.Select(y => cols.Select(x => new Point(x, y)).ToArray()).ToArray();

// Figure out whether you have time to search for the nest by calculating the
// area within the loop. How many tiles are enclosed by the loop?

var part2Answer = rowCols.Select(row => row.Aggregate(new Scanner(result2), (scanner, point) => scanner.ScanOne(pieceMap[point])).InsidePieces.Count()).Sum();
Console.WriteLine(part2Answer);

Inferred start shape is 7. Will replace
Found pipe entry at Piece { point = Point { x = 93, y = 25 }, shape = | }
No more next piece!
455


In [20]:
// 455 is correct!
Ensure(455, part2Answer);